# GSD: Assessing ambiguous nts in PB nuclear genomes

Checks a collection of PacBio sequenced yeast nuclear genomes from [Yue et al 2017](https://www.ncbi.nlm.nih.gov/pubmed/28416820) ambiguous/gap-representing residues. Comparing to SGD reference sequence from [here](https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/) as well.

References for sequence data:  
- [Contrasting evolutionary genome dynamics between domesticated and wild yeasts.
Yue JX, Li J, Aigrain L, Hallin J, Persson K, Oliver K, Bergström A, Coupland P, Warringer J, Lagomarsino MC, Fischer G, Durbin R, Liti G. Nat Genet. 2017 Jun;49(6):913-924. doi: 10.1038/ng.3847. Epub 2017 Apr 17. PMID: 28416820](https://www.ncbi.nlm.nih.gov/pubmed/28416820)


- [Life with 6000 genes. Goffeau A, Barrell BG, Bussey H, Davis RW, Dujon B, Feldmann H, Galibert F, Hoheisel JD, Jacq C, Johnston M, Louis EJ, Mewes HW, Murakami Y, Philippsen P, Tettelin H, Oliver SG. Science. 1996 Oct 25;274(5287):546, 563-7. PMID: 8849441](https://www.ncbi.nlm.nih.gov/pubmed/8849441)

-----

## Preparation

Get scripts and sequence data necessary.



In [1]:
!pip install pyfaidx

  Running setup.py bdist_wheel for pyfaidx ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/54/a2/b4/e242e58d23b2808e191b214067880faa46cd2341f363886e0b
Successfully built pyfaidx


Get the genomes data by running these commands.

In [2]:
import pandas as pd
# Prepare for getting PacBio (Yue et al 2017 sequences)
#make a list of the strain designations
yue_et_al_strains = ["S288C","DBVPG6044","DBVPG6765","SK1","Y12",
                     "YPS128","UWOPS034614","CBS432","N44","YPS138",
                     "UFRJ50816","UWOPS919171"]
# Get & unpack the genome sequences from strains 
for s in yue_et_al_strains:
    !curl -LO http://yjx1217.github.io/Yeast_PacBio_2016/data/Nuclear_Genome/{s}.genome.fa.gz
    !gunzip -f {s}.genome.fa.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0   2170      0 --:--:-- --:--:-- --:--:--  2373
100 3687k  100 3687k    0     0   264k      0  0:00:13  0:00:13 --:--:-- 1025k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0   2870      0 --:--:-- --:--:-- --:--:--  2870
100 3387k  100 3387k    0     0  9080k      0 --:--:-- --:--:-- --:--:-- 14.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0   2870      0 --:--:-- --:--:-- --:--:--  2870
100 3348k  100 3348k    0     0  6224k      0 --:--:-- --:--:-- --:--:-- 8564k
  % Total    % Received % Xferd  Average Speed   Tim

In [3]:
# add identifiers to each `chr` so results for each strain clear later
chromosome_id_prefix = "chr"
def add_strain_id_to_description_line(file,strain_id):
    '''
    Takes a file and edits every description line to add 
    strain_id after the caret.
    
    Saves the fixed file
    '''
    import sys
    output_file_name = "temp.txt"
    # prepare output file for saving so it will be open and ready
    with open(output_file_name, 'w') as output_file:

        # read in the input file
        with open(file, 'r') as input_handler:
            # prepare to give feeback later or allow skipping to certain start
            lines_processed = 0

            for line in input_handler:
                lines_processed += 1
                if line.startswith(">"):
                    rest_o_line = line.split(">")
                    new_line = ">"+strain_id + rest_o_line[1]
                else:
                    new_line = line
                
                # Send text to output
                output_file.write(new_line)

    
    # replace the original file with edited
    !mv temp.txt {file}
    # Feedback
    sys.stderr.write("\n{} chromosome identifiers tagged.".format(file))

for s in yue_et_al_strains:
    add_strain_id_to_description_line(s+".genome.fa",s)


S288C.genome.fa chromosome identifiers tagged.
DBVPG6044.genome.fa chromosome identifiers tagged.
DBVPG6765.genome.fa chromosome identifiers tagged.
SK1.genome.fa chromosome identifiers tagged.
Y12.genome.fa chromosome identifiers tagged.
YPS128.genome.fa chromosome identifiers tagged.
UWOPS034614.genome.fa chromosome identifiers tagged.
CBS432.genome.fa chromosome identifiers tagged.
N44.genome.fa chromosome identifiers tagged.
YPS138.genome.fa chromosome identifiers tagged.
UFRJ50816.genome.fa chromosome identifiers tagged.
UWOPS919171.genome.fa chromosome identifiers tagged.

In [4]:
# Cannot simply use `S288C_reference_sequence_R64-2-1_20150113.fsa` at https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
# because want nuclear only sequences here to match nuclear ones I obtaiend from 
# https://yjx1217.github.io/Yeast_PacBio_2016/data/, 
# where they keep nuclear and genome separate
chr_nums = ["01","02","03","04","05","06","07","08","09","10","11",
    "12","13","14","15","16"]
g_fns = []
for chr_num in chr_nums:
    fn = "chr"+chr_num+".fsa"
    !curl -OL https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/{fn}
    g_fns.append(fn)
    # edit or replace description lines to work similar to PacBio
    tag = "SGD_REFchr"+chr_num+" "
    add_strain_id_to_description_line(fn,tag)
# zip all nuclear chromosomes together as one file as they are from https://yjx1217.github.io/Yeast_PacBio_2016/data/
!cat {" ".join(g_fns)} > SGD_REF.genome.fa
!rm chr*.fsa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228k  100  228k    0     0   453k      0 --:--:-- --:--:-- --:--:--  453k



chr01.fsa chromosome identifiers tagged.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  807k  100  807k    0     0  1654k      0 --:--:-- --:--:-- --:--:-- 1651k



chr02.fsa chromosome identifiers tagged.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  314k  100  314k    0     0   840k      0 --:--:-- --:--:-- --:--:--  838k


...


chr14.fsa chromosome identifiers tagged.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1083k  100 1083k    0     0  2281k      0 --:--:-- --:--:-- --:--:-- 2276k



chr15.fsa chromosome identifiers tagged.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  941k  100  941k    0     0  2087k      0 --:--:-- --:--:-- --:--:-- 2087k



chr16.fsa chromosome identifiers tagged.

*FOR RELATED WORK:For keeping contents consistent, when switching to comparing Yue et al, 2017 set to the 1011 genomes set, I should add the PacBio-sequences mitochondrial genomes back in because they are among the 1011 genomes set. Plan is to do this in `GSD: Assessing ambiguous nts in PB complete genomes.ipynb`.*

In [5]:
# Make a list of all `genome.fa` files, excluding `genome.fa.nhr` and `genome.fa.nin` and `genome.fansq`
# The excluding was only necessary because I had run some BLAST queries preiminarily in development. Normally, 
# it would just be the `.re.fa` at the outset. (But keeping because removal ability could be useful.)
fn_to_check = "genome.fa" 
genomes = []
import os
import fnmatch
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*'+fn_to_check):
        if not file.endswith(".nhr") and not file.endswith(".nin") and not file.endswith(".nsq") :
            genomes.append(file)
genomes

['DBVPG6044.genome.fa',
 'SGD_REF.genome.fa',
 'DBVPG6765.genome.fa',
 'CBS432.genome.fa',
 'YPS128.genome.fa',
 'UFRJ50816.genome.fa',
 'S288C.genome.fa',
 'SK1.genome.fa',
 'UWOPS034614.genome.fa',
 'UWOPS919171.genome.fa',
 'YPS138.genome.fa',
 'N44.genome.fa',
 'Y12.genome.fa']


Now you are prepared to analyze each PacBio-sequenced genome.

## Assessing the genomes in regards to stretches of N base calls

### Total number of Ns present

In [6]:
from pyfaidx import Fasta
query = "N"
#query = "X"
#query = "A" # CONTROL. should occur!
for g in genomes:
    chrs = Fasta(g)
    ambiguous = []
    for x in chrs:
        #print(x.name)
        if query in str(x):
            ambiguous.append(x.name)
    #print(ambiguous)
    if not ambiguous:
        print ("No ambiguous nucleotides or gaps in {}.".format(g))
    else:
        print("There are occurences of {} in {}.".format(query,g))

There are occurences of N in DBVPG6044.genome.fa.
No ambiguous nucleotides or gaps in SGD_REF.genome.fa.
There are occurences of N in DBVPG6765.genome.fa.
There are occurences of N in CBS432.genome.fa.
There are occurences of N in YPS128.genome.fa.
There are occurences of N in UFRJ50816.genome.fa.
There are occurences of N in S288C.genome.fa.
There are occurences of N in SK1.genome.fa.
There are occurences of N in UWOPS034614.genome.fa.
There are occurences of N in UWOPS919171.genome.fa.
There are occurences of N in YPS138.genome.fa.
There are occurences of N in N44.genome.fa.
There are occurences of N in Y12.genome.fa.


Another way to assess, just count all the letters present:

In [7]:
%%time
from pyfaidx import Fasta
import pandas as pd
import collections
nt_counts = {}
for g in genomes:
    strain_id = g.split(".genome.fa")[0]
    concatenated_seqs = ""
    chrs = Fasta(g)
    for x in chrs:
        #print(x.name)
        concatenated_seqs += str(x)
    nt_counts[strain_id] = collections.Counter(concatenated_seqs)
nt_count_df = pd.DataFrame.from_dict(nt_counts, orient='index').fillna(0)
nt_count_df["Total_nts"] = nt_count_df.sum(1)
def percent_calc(items):
    '''
    takes a list of two items and calculates percentage of first item
    within total (second item)
    '''
    return items[0]/items[1]
nt_count_df['% N'] = nt_count_df[['N','Total_nts']].apply(percent_calc, axis=1)
nt_count_df = nt_count_df.style.format({'Total_nts':'{:.2E}','% N':'{:.2%}'})

CPU times: user 15.3 s, sys: 0 ns, total: 15.3 s
Wall time: 15.6 s


In [8]:
nt_count_df

,A,G,T,C,N,Total_nts,% N
CBS432,3683966,2322732,3675728,2321545,17357,1.20E+07,0.14%
DBVPG6044,3689297,2285884,3677723,2287780,17357,1.20E+07,0.15%
DBVPG6765,3647252,2258080,3632750,2257749,17357,1.18E+07,0.15%
N44,3620330,2278564,3617409,2278029,17357,1.18E+07,0.15%
S288C,3752659,2323469,3738132,2325532,17357,1.22E+07,0.14%
SGD_REF,3730180,2309287,3718146,2313713,0,1.21E+07,0.00%
SK1,3719655,2307796,3707198,2311279,17357,1.21E+07,0.14%
UFRJ50816,3735782,2326900,3732052,2325192,17357,1.21E+07,0.14%
UWOPS034614,3620938,2245047,3622509,2241495,17357,1.17E+07,0.15%
UWOPS919171,3652729,2270324,3646729,2269479,17357,1.19E+07,0.15%


### Examining the number and size of stretches of Ns present

Make a dataframe of the data and also make a summary one with the say, top five(?) number instances, for each.

In [10]:
%%time
# count frequency of blocks of Ns in the genomes
import re
from collections import defaultdict
from pyfaidx import Fasta
import pandas as pd
import collections


min_number_Ns_in_row_to_collect = 3
pattern_obj = re.compile("N{{{},}}".format(min_number_Ns_in_row_to_collect), re.I)  # adpated from
# code worked out in `collapse_large_unknown_blocks_in_DNA_sequence.py`, which relied heavily on
# https://stackoverflow.com/a/250306/8508004

len_match_dict_by_strain = {}

#genomes = ["N44.genome.fa"]

for g in genomes:
    len_match_dict = defaultdict(int)
    strain_id = g.split(".genome.fa")[0]
    records = Fasta(g)
    for record in records:
        for m in pattern_obj.finditer(str(record)):
            len_match_dict[len(m.group())] += 1
    #before passing final dictionary for strain to
    #collection, add an entry of zero for size of 1 so the
    #strain with no stretches will be in final dataframe
    if not len_match_dict:
        len_match_dict[1]=0
    len_match_dict_by_strain[strain_id] = len_match_dict

#stretches_size_freq_df = pd.DataFrame.from_dict(len_match_dict_by_strain, orient='index').fillna(0)
#stretches_size_freq_df = pd.DataFrame.from_dict(len_match_dict_by_strain).fillna(0).stack().reset_index() # if wanted all stretches of size to show count even if none present
stretches_size_freq_df = pd.DataFrame.from_dict(len_match_dict_by_strain).stack().reset_index() 
stretches_size_freq_df.columns = ['stretch_size','strain','#_instances'] 
stretches_size_freq_df = stretches_size_freq_df[['strain','stretch_size','#_instances']]
stretches_size_freq_df = stretches_size_freq_df.sort_values(['stretch_size','#_instances'],ascending=[0,0]).reset_index(drop=True)

CPU times: user 3.79 s, sys: 0 ns, total: 3.79 s
Wall time: 3.81 s


Because of use of `%%time` in above cell, easiest to display resulting dataframe using next cell. 

In [11]:
stretches_size_freq_df

,strain,stretch_size,#_instances
0,DBVPG6044,17357,1.0
1,DBVPG6765,17357,1.0
2,CBS432,17357,1.0
3,YPS128,17357,1.0
4,UFRJ50816,17357,1.0
5,S288C,17357,1.0
6,SK1,17357,1.0
7,UWOPS034614,17357,1.0
8,UWOPS919171,17357,1.0
9,YPS138,17357,1.0


Turns out there is a block of N nucleotides 'masking' the rDNA locus of each PacBio sequences. (Terminology illustrated in point #4 [here](https://www.reddit.com/r/bioinformatics/comments/35mquv/question_counting_the_number_of_ns_in_the_chimp/) by kbradnam.)  That spans what is equivalent from 451574 to 468993 of chromosome XII in SGD reference sequence.  
Since those numbers span 17419 bp in length, I don't know why they used 17357 N's and not 17419 Ns masking the locus?  Just added about 60 extra to cover future possibilities of extra? Or just to represent there's isn't quite the same as that span in the rDNA locus which just arbitrarily represents two loci repeats?  
Lack of assembly for the rDNA locus was pointed out in the paper.

Plan is to add the PacBio-sequences mitochondrial genomes back in and do this all again in `GSD: Assessing ambiguous nts in PB complete genomes.ipynb` next in preparation for comparing to the 1011 *cerevisiae* genomes collection where mitochondrial sequences mixed in among the genome assemblies.

----